<center> Advanced Integration and Annotation of scRNA-seq Data Using scVI: Hyperparameter Tuning, Label Transfer, and Custom Reference Creation

# Introduction

In this notebook, data from 10X Genomics will be uploaded following the application of Harmony correction. The scVI model, a preferred integration method for scRNA-seq data, will be tested here. Additionally, scVI and scANVI will be applied to enable label transfer from the CZI scRNA-seq atlas, which contains six RNA-seq samples.

The procedure is divided into two parts. First, label transfer and the second will be integration.

First, the data will be uploaded. For those interested in exploring alternative label transfer methods, the `celltypist` model can be used. However, this notebook will focus on atlas-based label transfer. [GitHub reference](https://github.com/mousepixels/sanbomics_scripts/blob/main/sc2024/annotation_integration.ipynb)


### Label Transfer

In [1]:
import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [2]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scvi
import torch
import celltypist
from celltypist import models
from scvi.autotune import ModelTuner
from ray import tune

CUDA available: True


ModuleNotFoundError: Please install ['hyperopt', 'ray.tune'] to use this functionality.

In [ ]:
print("CUDA available:", torch.cuda.is_available())

#### Reference from CZI:

In [3]:
ref_name = "/data/kanferg/Sptial_Omics/playGround/Data/Breast_Cancer/ref/ccdb972d-6655-43ae-9ad8-f895bd893d8a.h5ad"
adata_ref_init = sc.read_h5ad(ref_name)
# select all the breast samples
adata_ref_tisueRM = adata_ref_init[adata_ref_init.obs['tissue']=='breast',:].copy()
# select all the protein_coding
adata_ref = adata_ref_tisueRM[:,adata_ref_tisueRM.var['feature_type']=='protein_coding'].copy()
adata_ref.var.index = adata_ref.var["feature_name"].astype(str).values
adata_ref

AnnData object with n_obs × n_vars = 34164 × 18626
    obs: 'condition', 'replicate', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'RNA_snn_res.0.8', 'seurat_clusters', 'labels_score', 'Order', 'Lane', 'Index', 'cancer', 'reference', 'flowcell', 'min_umis', 'min_genes', 'percent_mito', 'expected_cells', 'total_droplets', 'z_dim', 'z_layers', 'channel_id', 'labels_cl_unif_per_channel', 'filt_median_genes', 'filt_median_umi', 'pass', 'ccpm_id', 'htapp', 'sequenced', 'stage_at_diagnosis', 'metastatic_presentation', 'biopsy_days_after_metastasis', 'ER_primary', 'ER_biopsy', 'PR_primary', 'PR_biopsy', 'HER2_primary', 'HER2_biopsy', 'receptors_primary', 'receptors_biopsy', 'site_biopsy', 'histology_breast', 'histology_biopsy', 'sampleid', 'cnv_cors', 'cnv_cors_max', 'cnv_score', 'cnv_ref_score', 'cnv_score_norm', 'cnv_score_norm_norm', 'cnv_condition', 'cnv_score_norm_norm2', 'pam50_Basal_single', 'pam50_Her2_single', 'pam50_LumA_single', 'pam50_LumB_single', 'pam50_Normal_single', 'pam50_m

In [4]:
from anndata import AnnData
rdata = AnnData(adata_ref.X, obs={"CellType": adata_ref.obs["cell_type"].values,"nCount_RNA":adata_ref.obs["nCount_RNA"].values,'percent_mito':adata_ref.obs["percent_mito"]} , var ={"n_cells":adata_ref.var["n_cells"].values, "feature_name":adata_ref.var["feature_name"].astype(str).values} )
rdata.var.index = rdata.var["feature_name"].values
rdata

AnnData object with n_obs × n_vars = 34164 × 18626
    obs: 'CellType', 'nCount_RNA', 'percent_mito'
    var: 'n_cells', 'feature_name'

In [5]:
rdata.obs.groupby('CellType').size()

CellType
fibroblast                               2797
blood vessel endothelial cell            2816
T cell                                   1274
adipocyte                                 595
chondrocyte                                 1
macrophage                                639
plasma cell                                31
mature NK T cell                           25
malignant cell                          25685
blood vessel smooth muscle cell           300
endothelial cell of hepatic sinusoid        1
dtype: int64

In [6]:
remove_cell = ['endothelial cell of hepatic sinusoid','chondrocyte']
rdata = rdata[~rdata.obs['CellType'].isin(remove_cell), :]
rdata.obs.groupby('CellType').size()

CellType
fibroblast                          2797
blood vessel endothelial cell       2816
T cell                              1274
adipocyte                            595
macrophage                           639
plasma cell                           31
mature NK T cell                      25
malignant cell                     25685
blood vessel smooth muscle cell      300
dtype: int64

In [7]:
rdata = rdata[~rdata.obs.CellType.isna()]

In [ ]:
# needs row count data and celltype in ref dataset and query data set. also batch
rdata.obs['batch'] = 'ref'
rdata.obs['sample'] = 'refrance'
